In [15]:
%load_ext autoreload
%autoreload 2
import innvestigate
import tensorflow as tf
from pathlib import Path
import xarray as xr
from shared.training import split_data_on_participants, train_and_evaluate
from shared.normalization import *
from shared.generators import SAT1DataGenerator
import matplotlib.pyplot as plt
import numpy as np
from shared.data import SAT1_STAGES_ACCURACY
from keras.models import load_model

# tf.compat.v1.disable_eager_execution()
from mne.viz import plot_topomap
from mne.io import read_info
from collections import defaultdict
from shared.lstm_network import LSTM_network

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [17]:
positions = read_info(Path("data/sat1/preprocessed/processed_0001_epo.fif"))

In [18]:
keras_model = load_model("models/sentiment_model.hdf5")

In [20]:
model = tf.keras.models.load_model("models/lstm")

n_hidden = 256
embedding_dim = 30
n_classes = 5
weights = model.get_weights()
# weights.insert(3, np.zeros_like(weights[0]))
# weights.insert(4, np.zeros_like(weights[1]))
# weights.insert(5, np.zeros_like(weights[2]))
# weights[6] = np.append(weights[6], np.zeros_like(weights[6]), axis=0)


2023-10-23 13:54:55.461462: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-10-23 13:54:55.560828: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-10-23 13:54:55.566637: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-10-23 13:54:56.122362: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.


In [21]:
for weight in weights:
    print(weight.shape)
print(len(weights))

(30, 1024)
(256, 1024)
(1024,)
(30, 1024)
(256, 1024)
(1024,)
(512, 5)
(5,)
8


In [22]:
lrp_model = LSTM_network(n_hidden, embedding_dim, n_classes, weights=weights)

In [23]:
test_gen = SAT1DataGenerator(test_data)
test_item = test_gen.__getitem__(0)
test_item = test_item[0][0, :, :]
test_item = np.expand_dims(test_item, 0)
test_item.shape

(1, 147, 30)

In [25]:
predictions = model.predict(test_item)
print(np.argmax(predictions))
lrp = lrp_model.full_pass(test_item)
print(np.argmax(lrp[0].numpy()))

1/1 [==============================] - 0s 16ms/step
2
4


In [14]:
lrp[0]

<tf.Tensor: shape=(1, 5), dtype=float64, numpy=array([[-0.537052  ,  0.30001867, -1.45530712,  2.82691889, -2.29232346]])>

In [84]:
explanation, _ =  lrp_model.lrp(test_item)

In [85]:
explanation

<tf.Tensor: shape=(1, 147, 30), dtype=float64, numpy=
array([[[-1.91452425e+00, -4.02220065e-01, -1.65003839e-01, ...,
          4.36453673e-01,  1.25822941e+00, -1.91818635e+00],
        [-2.01447931e-01,  2.76592608e-03,  2.29935304e-01, ...,
         -5.24956475e-02,  4.07994259e-01, -6.25517130e-01],
        [-5.40137668e-03, -4.92411556e-02,  1.39664726e-01, ...,
         -1.03041936e-01,  6.52144343e-02, -7.78840725e-02],
        ...,
        [ 3.19593875e-03,  5.64516740e-03, -3.07608029e-03, ...,
         -2.92439637e-03, -4.95639715e-05,  1.67471654e-04],
        [ 6.88304750e-03,  1.17758975e-02, -1.12168269e-02, ...,
         -2.04791926e-02, -1.49691763e-02,  4.47918406e-03],
        [ 1.98921186e+00, -3.88329332e-01,  3.01657218e+00, ...,
         -8.72399564e-01,  1.11689727e-01, -2.96677539e-01]]])>

In [34]:
# np.argmax(lrp_model.full_pass(test_item)[0], axis=1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3])